In [1]:
import bpy
import numpy as np
from IPython.display import display, Image
import os
import mathutils
from tqdm import tqdm
from typing import List, Dict
import bmesh
import cmath
import scipy.sparse
import scipy.linalg
from scipy.sparse.linalg import spsolve

import matplotlib.pyplot as plt

from line_profiler import profile

# install packages : "C:\Users\Pierre.Gilibert\OneDrive - ARVERNE\Bureau\blender-4.3.1-windows-x64\4.3\python\bin\python.exe" -m pip install scipy 

In [2]:
FOLDER = "C:\\Users\\pierr\\Documents\\Blender\\GeometryProcessing_Python\\"
FOLDER = "C:\\Users\\Pierre.Gilibert\\OneDrive - ARVERNE\\Documents\\Divers\\blender"
bpy.ops.wm.open_mainfile(filepath=os.path.join(FOLDER, "blender_notebook_v01.blend"))

{'FINISHED'}

In [3]:

class HEEdge:
    def __init__(self, edge, face, orientation:int):
        self.edge = edge
        self.face = face
        self.orientation = orientation
        self.vertex = edge.verts[orientation] # orientation in {0,1} stating whether the self.edge is in the same (0) direction as self or the opposite (1) direction.
        self.next = None
        self.twin = None
        self.angle_from_X = 0.0
        self.transport_coeff = 0.0
        self.vector_in_face =0 + 0j
        self.transport_complex = 0 + 0j

    def __str__(self):
        return f"HEedge : {self.edge} - orientation: {self.orientation}"
    
    def __repr__(self):
        return f"HEedge : {self.edge} - orientation: {self.orientation}"

    def __getattr__(self, item):
        # Delegate attribute access to the internal bmesh instance
        return getattr(self.edge, item)
    


In [7]:

class MYMesh:
    def __init__(self):
        self.bm = bmesh.new()
        self.facecorners = []
        self.vert2facecorner = {}
        self.facecorner_attributes = {}             # facecorners attributes
        self.vertex_attributes = {"u":{}, "v":{}}   # u,v : basis vector of the tangent plane at each vertex
        self.edge_attributes = {}
        self.face_attributes = {}
        self.face_vertex_to_facecorner = {}
        self.co = None                              # A (|V|, 3) np array where row i contains the x,y,z coordinates of the vertex indexed by i 
        self.cotan = None                           # A (3|F|,) np array where entry i is the cotangent of the angle of facecorner (loop) i
        self.face_areas = None                      # A (|F|,) np array where entry i is the area for face i
        self.internal_angles = None                 # A (3|F|,) np array where entry i is the angle of facecorner (loop) i
        self.fv = None                              # A (|F|, 3) np array where row i contains the indices of the vertices of face i
        self.heedges = []                           # List of Half edges
        self.dict_vert2heedges = {}


    def from_mesh(self, mesh_data):
        """ Mimic the bmesh from_mesh function. """
        self.bm.from_mesh(mesh_data)
        self.vert2facecorner = {v:[] for v in self.verts}
        self.facecorners = []
        self.facecorner_attributes["area"] = {}
        for face in self.faces:
            self.face_vertex_to_facecorner[face] = {}
            for loop in face.loops:
                self.vert2facecorner[loop.vert].append(loop)
                self.facecorners.append(loop)
                self.facecorner_attributes["area"][loop] = 0 # init 0 area
                self.face_vertex_to_facecorner[face][loop.vert] = loop
            
        self.fv = np.array([[fc.vert.index for fc in f.loops] for f in self.faces], dtype=int) # face #f has vertex [vi, vj, vk]
        self.co = np.array([v.co for v in self.verts])
        

    def to_mesh(self, mesh_data):
        """ Mimic the bmesh to_mesh function. """
        self.bm.to_mesh(mesh_data)
    
    # Add any other bmesh methods as needed:
    def free(self):
        """ Mimic the bmesh free function. """
        self.bm.free()

    # Add a custom method to directly access the internal bmesh:
    def __getattr__(self, item):
        # Delegate attribute access to the internal bmesh instance
        return getattr(self.bm, item)
    
    def ensure_lookup_tables(self):
        self.verts.ensure_lookup_table()
        self.edges.ensure_lookup_table()
        self.faces.ensure_lookup_table()

    def create_halfedge_datastructure(self):
        self.dict_vert2heedges = {}
        # self.edge_attributes["heedges"] = {}
        # for e in self.edges:
        #     self.edge_attributes["heedges"][e] = []
        for f in self.faces:
            if len(f.verts) != 3:
                raise ValueError("Not a triangular mesh : triangulate it beforehand !!")
            v_orientation = [f.verts[0].index, f.verts[1].index, f.verts[2].index]
            face_heedges = []
            for e in f.edges:
                # e0, e1, e2 = f.edges[0], f.edges[1], f.edges[2]
                v0, v1 = v_orientation.index(e.verts[0].index), v_orientation.index(e.verts[1].index)
                orientation = 1 - ((abs(v0-v1) == 1 and v0 < v1) or (abs(v0-v1) == 2 and v0 > v1)) # 0 if the edge matches the ccw convention of the face, 1 otherwise
                # print(e, v_orientation.index(e.verts[0].index), v_orientation.index(e.verts[1].index), orientation)
                face_heedges.append(HEEdge(e, f, orientation))
            he0, he1, he2 = face_heedges
            he0.next = he1
            he1.next = he2
            he2.next = he0
            self.dict_vert2heedges[(he0.vertex.index, he1.vertex.index)] = he0
            self.dict_vert2heedges[(he1.vertex.index, he2.vertex.index)] = he1
            self.dict_vert2heedges[(he2.vertex.index, he0.vertex.index)] = he2
            self.heedges.append(he0)
            self.heedges.append(he1)
            self.heedges.append(he2)
            # self.edge_attributes["heedges"][he0.edge] = he0
            # self.edge_attributes["heedges"][he1.edge] = he1
            # self.edge_attributes["heedges"][he2.edge] = he2

        for k in self.dict_vert2heedges:
            i0, i1 = k
            if i0 < i1:
                continue
            kr = (i1, i0)
            if kr in self.dict_vert2heedges:
                self.dict_vert2heedges[k].twin = self.dict_vert2heedges[kr]
                self.dict_vert2heedges[kr].twin = self.dict_vert2heedges[k]
        
        boundary_hedges = []
        for e in self.edges:
            if not e.is_boundary:
                continue
            i0, i1 = e.verts[0].index, e.verts[1].index
            if (i0, i1) in self.dict_vert2heedges:
                existing_he = self.dict_vert2heedges[(i0, i1)]
            elif (i1, i0) in self.dict_vert2heedges:
                existing_he = self.dict_vert2heedges[(i1, i0)]
            else:
                raise ValueError(f"Unable to find an half edge between vertices {i0} and {i1}")
            new_he = HEEdge(e, None, 1-existing_he.orientation)
            existing_he.twin = new_he
            new_he.twin = existing_he
            boundary_hedges.append(new_he)
            self.dict_vert2heedges[(new_he.vertex.index, e.other_vert(new_he.vertex).index)] = new_he
            self.heedges.append(new_he)

        # match the boundary hedges
        num_link_to_create = len(boundary_hedges)
        num_link_created = 0
        while num_link_created != num_link_to_create:
            he = boundary_hedges[num_link_created]
            # other_index = -1
            for i, ohe in enumerate(boundary_hedges):
                if ohe.vertex == he.twin.vertex:
                    other_index = i
                    break
            # ohe = boundary_hedges.pop(other_index)
            he.next = ohe
            num_link_created+=1


        # verification : all hedges should have a twin and a next defined
        for k in self.dict_vert2heedges:
            he = self.dict_vert2heedges[k]
            if he.twin is None or he.next is None:
                raise ValueError(f"Unable to define a twin or a next for hedge {k} : {he}")


    def _calculate_corner_area(self):
        nfaces = len(self.faces)
        ffc = np.arange(3*nfaces).reshape((nfaces, 3)) # face corner index : face #f has corners [i, j, k];
        angles = self.internal_angles[ffc] # array of alpha_i, alpha_j, alpha_k
        eij = self.co[self.fv[:,1]] - self.co[self.fv[:,0]] # edges
        ejk = self.co[self.fv[:,2]] - self.co[self.fv[:,1]]
        eki = self.co[self.fv[:,0]] - self.co[self.fv[:,2]]
        lij2 = eij[:,0]*eij[:,0] + eij[:,1]*eij[:,1] + eij[:,2]*eij[:,2] # squared lengths of the edges
        ljk2 = ejk[:,0]*ejk[:,0] + ejk[:,1]*ejk[:,1] + ejk[:,2]*ejk[:,2]
        lki2 = eki[:,0]*eki[:,0] + eki[:,1]*eki[:,1] + eki[:,2]*eki[:,2]

        # in case triangle i,j,k is non obtuse (all its angles are < pi/2) then this is the area of each face corner
        non_obtuse_area_i = 0.125 * (lij2 * self.cotan[ffc[:,2]] + lki2 * self.cotan[ffc[:,1]])
        non_obtuse_area_j = 0.125 * (ljk2 * self.cotan[ffc[:,0]] + lij2 * self.cotan[ffc[:,2]])
        non_obtuse_area_k = 0.125 * (lki2 * self.cotan[ffc[:,1]] + ljk2 * self.cotan[ffc[:,0]])

        facecorners_areas = np.zeros(3*nfaces).reshape((nfaces, 3))
        # check whether the angles are less than pi/2
        small_angle_bool = angles < np.pi/2
        big_angle_bool = ~small_angle_bool
        # True for all corners of triangle i,j,k ==> use the non_obtuse_area
        non_obtuse_bool = np.logical_and(small_angle_bool[:,0], np.logical_and(small_angle_bool[:,1], small_angle_bool[:,2]))
        facecorners_areas[non_obtuse_bool, 0] = non_obtuse_area_i[non_obtuse_bool]
        facecorners_areas[non_obtuse_bool, 1] = non_obtuse_area_j[non_obtuse_bool]
        facecorners_areas[non_obtuse_bool, 2] = non_obtuse_area_k[non_obtuse_bool]
        # False for corner p in {i,j,k}: use half the face area for p and a quarter for the two others corners
        facecorners_areas[big_angle_bool[:,0]] = self.face_areas[big_angle_bool[:,0]][:,None] * np.array([[0.5, 0.25, 0.25]])
        facecorners_areas[big_angle_bool[:,1]] = self.face_areas[big_angle_bool[:,1]][:,None] * np.array([[0.25, 0.5, 0.25]])
        facecorners_areas[big_angle_bool[:,2]] = self.face_areas[big_angle_bool[:,2]][:,None] * np.array([[0.25, 0.25, 0.5]])
        self.facecorner_attributes["area"] = facecorners_areas.ravel()

    def _calculate_corner_angles_and_face_areas(self):

        vi = self.co[self.fv[:,0]]
        vj = self.co[self.fv[:,1]]
        vk = self.co[self.fv[:,2]]

        eij, ejk, eki = vj-vi, vk-vj, vi-vk
        lij2 = eij[:,0]*eij[:,0] + eij[:,1]*eij[:,1] + eij[:,2]*eij[:,2]
        ljk2 = ejk[:,0]*ejk[:,0] + ejk[:,1]*ejk[:,1] + ejk[:,2]*ejk[:,2]
        lki2 = eki[:,0]*eki[:,0] + eki[:,1]*eki[:,1] + eki[:,2]*eki[:,2]
        lij, ljk, lki = np.sqrt(lij2), np.sqrt(ljk2), np.sqrt(lki2)

        s = 0.5 * (lij + ljk + lki) # half perimeter of every triangle
        self.face_areas = 0.5 * np.sqrt(s * (s - lij) * (s - ljk) * (s - lki)) # Heron's formula for the area of the triangles
        
        q_i = -ljk2 + lij2 + lki2
        q_j = -lki2 + ljk2 + lij2
        q_k = -lij2 + lki2 + ljk2

        denom_inv = 1/(8*self.face_areas)
        self.cotan = np.zeros(3*len(bm.faces))
        self.cotan[0::3] = q_i*denom_inv
        self.cotan[1::3] = q_j*denom_inv
        self.cotan[2::3] = q_k*denom_inv

        self.internal_angles = np.zeros(3*len(bm.faces))
        self.internal_angles[0::3] = np.arccos(np.clip(q_i / (2*lij * lki), -1, 1))
        self.internal_angles[1::3] = np.arccos(np.clip(q_j / (2*ljk * lij), -1, 1))
        self.internal_angles[2::3] = np.arccos(np.clip(q_k / (2*lki * ljk), -1, 1))

    def calculate_halfedge_vector_in_face(self):
        edge_indices = np.array([[e.verts[0].index, e.verts[1].index] for e in self.edges])
        edge_dir = bm.co[edge_indices[:,1]] - bm.co[edge_indices[:,0]]
        edge_square_len = edge_dir[:,0]*edge_dir[:,0] + edge_dir[:,1]*edge_dir[:,1] + edge_dir[:,2]*edge_dir[:,2]
        edge_len = np.sqrt(edge_square_len)

        heAB:HEEdge
        heBC:HEEdge
        heCA:HEEdge
        for f in self.faces:
            area = self.face_areas[f.index]

            heAB = self.dict_vert2heedges[(f.verts[0].index, f.verts[1].index)]
            heBC = heAB.next
            heCA = heBC.next

            lAB = edge_len[heAB.edge.index]
            lBC = edge_len[heBC.edge.index]
            lCA = edge_len[heCA.edge.index]

            # pA = [0,0]
            # pB = np.array([lAB, 0])
            pB = lAB + 0j

            h = 2*area / lAB
            w = np.sqrt(np.maximum(0, lCA*lCA - h*h))
            if (lBC * lBC > (lAB * lAB + lCA * lCA)):
                w *= -1

            # pC = np.array([w, h])
            pC = w + 1j*h
            heAB.vector_in_face = pB
            heBC.vector_in_face = pC - pB
            heCA.vector_in_face = -pC

    def calculate_transport_complex_along_halfedge(self):

        heA:HEEdge
        heB:HEEdge
        for e in self.edges:
            if e.is_boundary:
                continue
            heA = self.dict_vert2heedges[(e.verts[0].index, e.verts[1].index)]
            heB = heA.twin

            zA = heA.vector_in_face
            zB = heB.vector_in_face
            rot = -zB/zA
            rot /= np.absolute(rot)

            heA.transport_complex = rot
            heB.transport_complex = np.conj(rot)

    def compute_edge_cotan_weight(self):

        edge_indices = np.array([[e.verts[0].index, e.verts[1].index] for e in self.edges])
        edge_dir = bm.co[edge_indices[:,1]] - bm.co[edge_indices[:,0]]
        edge_square_len = edge_dir[:,0]*edge_dir[:,0] + edge_dir[:,1]*edge_dir[:,1] + edge_dir[:,2]*edge_dir[:,2]
        edge_len = np.sqrt(edge_square_len)

        edge_cotan_weight = np.zeros(len(self.edges))

        for e in self.edges:
            cot_sum = 0
            heA = self.dict_vert2heedges[(e.verts[0].index, e.verts[1].index)]
            heB = heA.twin
            for he in [heA, heB]:
                if he.face is None:
                    continue
                
                l_ij = edge_len[he.edge.index]
                he = he.next
                l_jk = edge_len[he.edge.index]
                he = he.next
                l_ki = edge_len[he.edge.index]
                area = self.face_areas[he.face.index]
                cot_value = (-l_ij * l_ij + l_jk * l_jk + l_ki * l_ki) / (4. * area)
                cot_sum += 0.5 * cot_value
            edge_cotan_weight[e.index] = cot_sum

        self.edge_attributes["cotan_weight"] = edge_cotan_weight

    
    def compute_vertex_connection_laplacian(self, n):
        he:HEEdge
        row, col, val = [],  [], []
        for k in self.dict_vert2heedges.keys():
            he = self.dict_vert2heedges[k]

            i_tail = he.vertex.index
            i_head = he.next.vertex.index

            rot = np.power(he.twin.transport_complex, n)
            weight = self.edge_attributes["cotan_weight"][he.edge.index]
            row.extend([i_tail, i_tail])
            col.extend([i_tail, i_head])
            val.extend([weight, - weight * rot])

        matrix = scipy.sparse.coo_array((val, (row, col))).tocsr() + 1e-9 * scipy.sparse.eye(len(self.verts))
        return matrix
    

    def compute_vertex_galerkin_mass_matrix(self):
        row, col, val = [], [], []
        for f in self.faces:
            area = self.face_areas[f.index]
            vi, vj, vk = f.verts
            i,j,k = vi.index, vj.index, vk.index

            a6 = area / 6.0
            a12 = area / 12.0

            row.extend([i,  j,  k,   i,   i,   j,   j,   k,   k])
            col.extend([i,  j,  k,   j,   k,   i,   k,   i,   j])
            val.extend([a6, a6, a6,  a12, a12, a12, a12, a12, a12])

        matrix = scipy.sparse.coo_array((val, (row, col))).tocsr()
        return matrix
        







obj = bpy.data.objects["Suzanne.001"]

bpy.ops.object.mode_set(mode='OBJECT')
mesh = obj.data
bm = MYMesh()
bm.from_mesh(mesh)
bm.ensure_lookup_tables()
bm.create_halfedge_datastructure()
bm._calculate_corner_angles_and_face_areas()
bm.calculate_halfedge_vector_in_face()
bm.calculate_transport_complex_along_halfedge()
bm.compute_edge_cotan_weight()



In [8]:

def smallest_eigenvector_square(energy_matrix, mass_matrix, n_iterations):
    """
    Compute the smallest eigenvector of the square using an iterative method.

    Parameters:
    - energy_matrix: Sparse energy matrix (scipy.sparse.csr_matrix)
    - mass_matrix: Sparse mass matrix (scipy.sparse.csr_matrix)
    - n_iterations: Number of iterations for the method.

    Returns:
    - x: Approximation of the smallest eigenvector.
    """
    N = energy_matrix.shape[0]  # Number of rows
    u = np.random.rand(N)       # Random initial vector
    x = u.copy()                # Initialize x

    for _ in range(n_iterations):
        # Solve the linear system: energy_matrix * x = mass_matrix * u
        x = spsolve(energy_matrix, mass_matrix @ u)

        # Re-normalize x with respect to the mass matrix
        x = normalize(x, mass_matrix)

        # Update u for the next iteration
        u = x.copy()

    return x

def normalize(x, mass_matrix):
    """
    Normalize vector x with respect to the mass matrix.
    """
    norm = np.sqrt(x @ (mass_matrix @ x))  # Quadratic form norm
    return x / norm if norm != 0 else x



def smoothest_vertex_direction_field(self:MYMesh, n):
    mass_matrix = self.compute_vertex_galerkin_mass_matrix()
    energy_matrix = self.compute_vertex_connection_laplacian(n)

    

    


In [9]:
mass_matrix = bm.compute_vertex_galerkin_mass_matrix()
mass_matrix = 0.5 * (mass_matrix + mass_matrix.T)
energy_matrix = bm.compute_vertex_connection_laplacian(n=1)
energy_matrix = energy_matrix + 1e-8 * mass_matrix
m_inv = scipy.sparse.linalg.inv(mass_matrix)
matrix_A = m_inv @ energy_matrix
# solution = smallest_eigenvector_square(energy_matrix, mass_matrix, 10)


In [10]:
eigenvalue, eignevector = scipy.sparse.linalg.eigs(matrix_A, k=1, which="SM")

In [12]:

all_X = []
for i, v in enumerate(bm.verts):
    he = bm.vertex_attributes["X"][i]
    X = np.array(he.twin.vertex.co - he.vertex.co)
    X_perp = np.dot(X, v.normal)
    X = X - X_perp * v.normal
    all_X.append(X/np.linalg.norm(X))

all_X = np.array(all_X)

key = "SM"

u = eignevector
a = (np.angle(u.ravel())/1) 
field = np.cos(a) + 1j * np.sin(a)




if 'X' in mesh.attributes:
    mesh.attributes.remove(mesh.attributes["X"])
    mesh.attributes.remove(mesh.attributes["angle"])
    
attr = mesh.attributes.new(name="X", type='FLOAT_VECTOR', domain='POINT')
attr.data.foreach_set('vector', all_X.flatten())

attr = mesh.attributes.new(name="angle", type='FLOAT', domain='POINT')
attr.data.foreach_set('value', a.flatten())
    

KeyError: 'X'